In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import shutil
from urllib.parse import urljoin, urlparse, parse_qs

urls = {

    "Wood"   : "https://www.czepeku.com/fantasy/maps/search/clearing+forest+jungle+swamp+tree",
    "Pasture": "https://www.czepeku.com/fantasy/maps/search/farm+garden+meadow",
    "Volcano": "https://www.czepeku.com/fantasy/maps/search/fire+infernal+lava+volcano",
    "Snow"   : "https://www.czepeku.com/fantasy/maps/search/frozen+snow",
    "City"   : "https://www.czepeku.com/fantasy/maps/search/camp+fortification+market+urban+village"
}


def get_images_url(url, key):

    page   = requests.get(url)
    soup   = BeautifulSoup(page.content, 'html.parser')
    images = soup.find_all('img')

    os.makedirs(f'../data/images/{key}', exist_ok=True)
    counter = 1

    for image in images:

        relative_url  = image.get('src')
        absolute_url  = urljoin(url, relative_url)
        parsed_url    = urlparse(absolute_url)
        query_params  = parse_qs(parsed_url.query)
        image_name    = f'image_{counter - 3}.jpg'
        counter      += 1
        image_path    = os.path.join(f'../data/images/{key}', image_name)
        response      = requests.get(absolute_url)
        
        if counter <= 4:
            continue

        if response.status_code == 200:
            with open(image_path, 'wb') as f:
                f.write(response.content)
        
    print('Done')


In [ ]:

target_dir = '../data/images'

if os.path.exists(target_dir):
    shutil.rmtree(target_dir)

for key in urls:
    get_images_url(urls[key], key)

In [ ]:
import os
from PIL import Image

def resize_images_in_folder(root_folder, target_size=(1024, 1024)):
    for dirpath, dirnames, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                file_path = os.path.join(dirpath, filename)
                try:
                    with Image.open(file_path) as img:
                        img_resized = img.resize(target_size)
                        img_resized.save(file_path)
                        print(f"Resized: {file_path}")
                except Exception as e:
                    print(f"Failed to resize {file_path}: {e}")

resize_images_in_folder('../../data/images/')
